In [1]:
import pandas as pd
import time

In [4]:
def basic_set(df):
    basic = {}
    for i in df.drop_duplicates().values.tolist():
        basic[str(i)] = []
        for j, k in enumerate(df.values.tolist()):
            if k == i:
                basic[str(i)].append(j)

    return basic

def rough_set(data,y_name):
    data = data.dropna(axis=0, how='any')
    x_data = data.drop([y_name], axis=1)
    y_data = data[y_name]
    # 决策属性基本集
    y_basic_set = sorted([v for k, v in basic_set(y_data).items()])
    # 条件属性基本集
    x_basic_set = sorted([v for k, v in basic_set(x_data).items()])
    pos = []
    for i in x_basic_set:
        for j in y_basic_set:
            if set(i).issubset(j):
                pos.append(i)
    pos.sort()
#     print("x_basic_set",x_basic_set)
#     print("y_basic_set",y_basic_set)
#     print ('y的x正域Pos_x(y): ',pos)
    r_x_y = len([k for i in pos for k in i]) / (len(data))
    # print('依赖度r_x_(y):', r_x_y)

    # 探索条件属性中不可省关系
    columns_num = list(range(x_data.shape[1])) 
    
    # 收集核
    imp_core = []
    
    # 收集属性重要性
    imp_attr = []
    
    for i in columns_num:
        c = columns_num.copy()
        c.remove(i)
        u = data.iloc[:, c]
        u = sorted([v for k, v in basic_set(u).items()])
        pos_va = []
        
        for k in u:
            for j in y_basic_set:
                if set(k).issubset(j):
                    pos_va.append(k)
        
        if sorted(pos_va) != pos: 
            """
            如果去掉了了这个之后两个正域不相等，则是重要属性，放进imp_core
            """
            imp_core.append(i)
        
        r = len(sorted(pos_va)) / len(data)
        r_diff = round(r_x_y - r, 4)

        imp_attr.append(r_diff)

    dict_imp = {}
    for o, p in enumerate(imp_attr):
        dict_imp[data.columns[o]] = p

    result = dict_imp
    sorted_dict_imp = sorted(dict_imp, key=lambda x: dict_imp[x], reverse=True)
    sorted_dict_imp = list(map(lambda x: {x: dict_imp[x]}, sorted_dict_imp))
    imp_core = [data.columns[i] for i in imp_core]

    print('属性重要度(越大越重要)为:', sorted_dict_imp)
    print("imp_core:{}".format(imp_core))
    return dict_imp,sorted_dict_imp,imp_core

def deal(data,y_name):
    #获取数据长度
    len1 = data.shape[0]
    
    sorted_dict_imp = [[]]*len1
    total = [0]*len1
    title = data.columns.tolist()[:-1]
    dict_imp,sorted_dict_imp,imp_core = rough_set(data,y_name)
    return dict_imp,sorted_dict_imp,imp_core

In [34]:
time1 = time.time()
#读取文件数据
data = pd.read_csv(filepath_or_buffer='/Users/sijianxuan/Documents/test2.csv')
dict_imp,sorted_dict_imp,imp_core = deal(data,"salary")
time2 = time.time()
print("runnning time is {}s".format(round(time2-time1,2)))

属性重要度(越大越重要)为: [{'education': 0.5}, {'age': 0.3889}, {'sex': 0.3889}]
imp_core:['education', 'age', 'sex']
runnning time is 0.07s


In [86]:
test= data.sort_values(by=["age","education","sex","salary"])
test1 = pd.DataFrame(test.groupby(["age","education"])["salary"].value_counts(normalize=True))
test1.columns = ["frequency"]
# test1.reset_index(inplace=True)
test1.frequency=(round(test1["frequency"]*100,2)).astype(str)+"%"
print(test1)

                     frequency
age education salary          
0   0         0         100.0%
    1         1         100.0%
1   1         1         55.56%
              0         44.44%


In [87]:
test= data.sort_values(by=["age","education","sex","salary"])
test1 = pd.DataFrame(test.groupby(["age","education","sex"])["salary"].value_counts(normalize=True))
test1.columns = ["frequency"]
# test1.reset_index(inplace=True)
test1.frequency=(round(test1["frequency"]*100,2)).astype(str)+"%"
print(test1)

                         frequency
age education sex salary          
0   0         0   0         100.0%
              1   0         100.0%
    1         0   1         100.0%
              1   1         100.0%
1   1         0   0          50.0%
                  1          50.0%
              1   1          60.0%
                  0          40.0%
